In [3]:
import numpy as np
import pandas as pd
import os
import datetime as dt

In [4]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [5]:
from sklearn.model_selection import train_test_split

In [2]:
from library.load_data_function import load_data
from library.model_training import model_training

In [6]:
import tensorflow as tf

print(tf.__version__)


tf.config.list_physical_devices('GPU') #2.xx 부터는 gpu_available 대신 이렇게 하라고 한다

2.5.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
data_name = ['gear_wear', '3W', 'smartphone', 'condition_monitoring']

In [8]:
data_type = data_name[0]

In [9]:
new_data_set, label, features = load_data(data_type)

In [10]:
#features = ['gyro_x', 'gyro_y', 'gyro_z', 'accelerate_x', 'accelerate_y', 'accelerate_z', 'current_4', 'current_5', 'current_6']
#features = ['gyro_x', 'gyro_y', 'gyro_z', 'accelerate_x', 'accelerate_y', 'accelerate_z']
#features = ['P-PDG','P-TPT','T-TPT','P-MON-CKP']
#features = ['attitude.roll', 'attitude.pitch', 'attitude.yaw', 'gravity.x', 'gravity.y', 'gravity.z', 'rotationRate.x', 'rotationRate.y', 'rotationRate.z', 'userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
total = []

for i in range(len(new_data_set)):
    df_original = pd.DataFrame(data = new_data_set[i], columns = features)
    total.append(df_original)

total_one_p = total[0:56231]
total_two_p = total[56232:]

state = []
info_one_p = []
info_two_p = []
one_idx = 0
two_idx = 0

for i in range(len(label)):
    if label[i] == 0:
        one_idx += 1
        state.append('wear')
        info_one_p.append('wear_idx'+str(one_idx))
    elif label[i] == 1:
        two_idx += 1
        state.append('normal')
        info_two_p.append('normal_idx'+str(two_idx))

info = np.concatenate((info_one_p, info_two_p))

data_dict = {}

for i in range(len(info)):
    data_dict[info[i]] = total[i]

test_case = 1

min_length = np.min([len(total_one_p),len(total_two_p)])

if test_case == 1:
    wear_length = int(min_length * 0.2)
elif test_case == 2:
    wear_length = int(min_length * 0.4)
elif test_case == 3:
    wear_length = int(min_length * 0.6)
elif test_case == 4:
    wear_length = int(min_length * 0.8)
elif test_case == 5:
    wear_length = int(min_length * 1.0)

total_one = total_one_p[0:wear_length]
total_two = total_two_p[0:min_length]

info_one = info_one_p[0:wear_length]
info_two = info_two_p[0:min_length]

label_one = np.array([0 for i in range(wear_length)])
label_two = np.array([1 for i in range(min_length)])

total_balance = np.concatenate((total_one, total_two))
info_balance = np.concatenate((info_one, info_two))
label_balance = np.concatenate((label_one,label_two))

state_balance = []
for i in label_balance:
    if i == 0:
        state_balance.append('wear')
    elif i == 1:
        state_balance.append('normal')

x_train_one_id, x_test_one_id, y_train_one, y_test_one = train_test_split(info_one, label_one, test_size=0.2, shuffle=True, random_state=10)
x_train_two_id, x_test_two_id, y_train_two, y_test_two = train_test_split(info_two, label_two, test_size=0.2, shuffle=True, random_state=10)

x_valid_one_id, x_test_one_id, y_valid_one, y_test_one = train_test_split(x_test_one_id, y_test_one, test_size=0.5, shuffle=True, random_state=10)
x_valid_two_id, x_test_two_id, y_valid_two, y_test_two = train_test_split(x_test_two_id, y_test_two, test_size=0.5, shuffle=True, random_state=10)

x_train_one = []
x_train_two = []

x_valid_one = []
x_valid_two = []

x_test_one = []
x_test_two = []


for i in range(len(x_train_one_id)):
    x_train_one.append(data_dict[x_train_one_id[i]])

for i in range(len(x_train_two_id)):
    x_train_two.append(data_dict[x_train_two_id[i]])

for i in range(len(x_valid_one_id)):
    x_valid_one.append(data_dict[x_valid_one_id[i]])
    
for i in range(len(x_valid_two_id)):
    x_valid_two.append(data_dict[x_valid_two_id[i]])
    
for i in range(len(x_test_one_id)):
    x_test_one.append(data_dict[x_test_one_id[i]])
    
for i in range(len(x_test_two_id)):
    x_test_two.append(data_dict[x_test_two_id[i]])
    

x_train = np.concatenate((x_train_one,x_train_two))
y_train = np.concatenate((y_train_one,y_train_two))

x_valid = np.concatenate((x_valid_one,x_valid_two))
y_valid = np.concatenate((y_valid_one,y_valid_two))

x_test = np.concatenate((x_test_one, x_test_two))
y_test = np.concatenate((y_test_one, y_test_two))

train_info = np.concatenate((x_train_one_id,x_train_two_id))
valid_info = np.concatenate((x_valid_one_id,x_valid_two_id))
test_info = np.concatenate((x_test_one_id,x_test_two_id))

train_state = []
valid_state = []
test_state = []

for i in y_train:
    if i == 0:
        train_state.append('wear')
    elif i == 1:
        train_state.append('normal')
        
for i in y_valid:
    if i == 0:
        valid_state.append('wear')
    elif i == 1:
        valid_state.append('normal')        

        
for i in y_test:
    if i == 0:
        test_state.append('wear')
    elif i == 1:
        test_state.append('normal')



In [11]:
now = dt.datetime.now()

MODEL_SAVE_FOLDER_PATH = "../Model/CNN_feature_model/"+str(data_type)+'_'+str(now.year)+str(now.month)+str(now.day)+'_'+str(now.hour)+str(now.minute)+str(now.second)+'/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'epoch_{epoch:02d}_{val_loss:.4f}.h5'
cd_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=0, save_best_only=True)

In [12]:
modelName = 'LITE'

train_model, train_history = model_training(modelName, x_train, y_train, x_valid, y_valid, 10, 16)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 6)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 60, 32)       224         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 60, 32)       608         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 60, 32)       992         input_1[0][0]                    
______________________________________________________________________________________________

KeyboardInterrupt: 